# Kristins Alex 

## Imports

In [1]:
import os
from datetime import datetime
from dataclasses import dataclass

import talos as ta
from talos.model import lr_normalizer
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

#gpu_options = tf.GPUOptions(allow_growth=True)
#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  1 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1682735129922965593
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 20264236482
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17476102872684193506
physical_device_desc: "device: 0, name: GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6"
]
1 Physical GPUs, 1 Logical GPUs


## AlexNet

In [2]:
def alexnet(activation, leaky_alpha, dropout):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation=activation_layer, input_shape=(224,224,Global.num_image_channels)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(units = 2, activation=activation_layer)
        #Dense(10, activation='softmax')
    ])
    return model

## VGG16

In [3]:
def vgg16(activation, leaky_alpha):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    
    model.add(Conv2D(input_shape=(224,224,Global.num_image_channels), filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(units = 4096, activation = activation_layer))
    model.add(Dense(units = 4096, activation = activation_layer))
    #model.add(Dense(units = 2, activation = "softmax"))
    model.add(Dense(units = 2, activation=activation_layer))

    #opt = Adam(lr = 0.001)
    #model.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy, metrics = ['accuracy'])
    #model.summary()
    return model

## Generation of the DNN

In [4]:
def gen_net(x_train, y_train, x_val, y_val, params):
    
    K.clear_session()
    
    train_generator, valid_generator = create_data_pipline(params['batch_size'], params['samples'])
    tg_steps_per_epoch = train_generator.n // train_generator.batch_size
    vg_validation_steps = valid_generator.n // valid_generator.batch_size
    print('Steps per Epoch: {}, Validation Steps: {}'.format(tg_steps_per_epoch, vg_validation_steps))
    
 
    if(Global.net_architecture == 'ALEX'):
        model = alexnet(params['activation'], params['leaky_alpha'], params['dropout'])
    elif(Global.net_architecture =='VGG16'):
        model = vgg16(params['activation'], params['leaky_alpha'])
    else:
        print('Wrong net architecture!')
            
    model.compile(
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer'])), 
        loss = Global.loss_function, 
        metrics = get_reduction_metric(Global.reduction_metric)
    )
    #print(model.summary())
    print('_________________________________________________________________')
    
    startTime = datetime.now()
    
    out = model.fit(
        x = train_generator,
        epochs = params['epochs'],
        validation_data = valid_generator,
        steps_per_epoch = tg_steps_per_epoch,
        validation_steps = vg_validation_steps,
        workers = 8
    )
    print("Time taken:", datetime.now() - startTime)

    return out, model

### Benutzerdefinierte Kostenfunktion & Metrik

In [5]:
def mean_absolut_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Hilfsfunktion

In [6]:
def get_reduction_metric(metric):
    
    if metric == 'mean_absolut_error':
        return [mean_absolut_error]
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Struct for global parameter

In [7]:
@dataclass
class global_parameter:
    loss_function: str = 'mean_squared_error'
    reduction_metric: str = 'mean_absolut_error'
    monitor_value: str = 'val_mean_absolut_error'
        
    net_architecture = 'VGG16' # 'ALEX' vs 'VGG16'
    
    dataset: str = '201129_2031'
    device: str = 'RTX_2080_Ti'
    data_augmentation: bool = True
    image_channels: str = 'rgb' # just change this, everything else will automaticlly adjusted
    num_image_channels: int = 3
    image_dir: str = '..\\..\\data_generation\\dataset\\{}\\'.format(dataset)
    
    csv_file_name: str = 'labels_ks_RGB.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(net_architecture, dataset)

        
Global = global_parameter

if(Global.image_channels == 'rgba'):
    Global.num_image_channels = 4
    csv_file_name: str = 'lables_ks_RGBD.csv'
    csv_file: str = image_dir + csv_file_name
    target_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    results: str = '\\..\\{}_{}_Results.csv'.format(net_architecture, dataset)

### Generierung Datenpipeline (Angepasst für Talos)

In [8]:
def create_data_pipline(batch_size, num_samples):
    
    
    df = pd.read_csv(Global.csv_file)
    df_shuffled = df.sample(frac = 1, random_state = 1)
    df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]
    df_valid = df_shuffled.drop(df_shuffled.index[0 : df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
    
    if Global.data_augmentation:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.1,
            brightness_range = (0.5, 1.0), 
            fill_mode = 'nearest'
        )
    else:
        train_data_generator = ImageDataGenerator(
            rescale = 1./255
        )
        
    train_generator = train_data_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = True,
        seed = 77,
        batch_size = batch_size
    )
        
    valid_data_generator = ImageDataGenerator(
        rescale = 1./255
    )
    
    valid_generator = valid_data_generator.flow_from_dataframe(
        dataframe = df_valid,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        class_mode = 'raw',
        target_size = (224, 224),
        color_mode = Global.image_channels,
        shuffle = False,
        seed = 77,
        batch_size = batch_size
    )
    
    return train_generator, valid_generator

In [9]:

if(not os.path.exists(Global.target_dir)):
    os.makedirs(Global.target_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(Global.target_dir))

device_file = open(Global.target_dir + '{}.txt'.format(Global.device), "a+")
device_file.close()

Directory >>| ..\output\VGG16_201129_2031_rgb\ |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)


### GridSerach Parameter

In [10]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad
#
#hyper_parameter = {
#    'samples': [20000],
#    'epochs': [1],
#    'batch_size': [32, 64],
#    'optimizer': [Adam],
#    'lr': [1, 2, 5],
#    'first_neuron': [1024, 2048, 4096],
#    'dropout': [0.25, 0.50],
#    'activation': ['leakyrelu', 'relu'],
#    'hidden_layers': [0, 1, 2, 3, 4],
#    'leaky_alpha': [0.1] #Default bei LeakyReLU, sonst PReLU
#}

hyper_parameter = {
    'samples': [20000],
    'epochs': [1],
    'batch_size': [16, 32],# 64],
    'optimizer': [Adam],
    'lr': [1, 2, 5],
    'first_neuron':  [1024, 2048, 4096],
    'dropout': [0.25, 0.50],
    'activation': ['leakyrelu', 'relu'],
    'hidden_layers': [4], #[0, 1, 2, 3, 4],
    'leaky_alpha': [0.1] #Default bei LeakyReLU, sonst PReLU
}

### Start Talos Search

In [11]:
dummy_x = np.empty((1, 2, 3, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    
        t = ta.Scan(
            x = dummy_x,
            y = dummy_y,
            model = gen_net,
            params = hyper_parameter,
            experiment_name = '{}'.format(Global.dataset),
            #shuffle=False,
            reduction_metric = Global.reduction_metric,
            disable_progress_bar = False,
            print_params = True,
            clear_session = True,
            save_weights = False
        )
        

#t.data.to_csv(Global.target_dir + Global.results, index = True)
t.data.to_csv(Global.target_dir + Global.results, index = True, mode='a', header=False)

  0%|                                                                                           | 0/72 [00:00<?, ?it/s]

{'activation': 'leakyrelu', 'batch_size': 16, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 4, 'leaky_alpha': 0.1, 'lr': 1, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 1000, Validation Steps: 250
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1000 steps, validate for 250 steps
1000/1000 [==============================] - 1145s 1s/step - loss: nan - mean_absolut_error: nan - val_loss: nan - val_mean_absolut_error: nan


  1%|█                                                                             | 1/72 [19:07<22:37:55, 1147.55s/it]

Time taken: 0:19:05.544419
{'activation': 'leakyrelu', 'batch_size': 16, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 4, 'leaky_alpha': 0.1, 'lr': 2, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 1000, Validation Steps: 250
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1000 steps, validate for 250 steps
1000/1000 [==============================] - 198s 198ms/step - loss: nan - mean_absolut_error: nan - val_loss: nan - val_mean_absolut_error: nan


  3%|██▏                                                                            | 2/72 [22:26<16:46:56, 863.09s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████▊  | 70/72 [4:05:27<06:26, 193.13s/it]

{'activation': 'relu', 'batch_size': 32, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 4096, 'hidden_layers': 4, 'leaky_alpha': 0.1, 'lr': 2, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 500, Validation Steps: 125
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 500 steps, validate for 125 steps
500/500 [==============================] - 191s 382ms/step - loss: 6606.6004 - mean_absolut_error: 58.9816 - val_loss: 5816.9313 - val_mean_absolut_error: 56.5364
Time taken: 0:03:11.681677


 99%|█████████████████████████████████████████████████████████████████████████████▉ | 71/72 [4:08:40<03:13, 193.03s/it]

{'activation': 'relu', 'batch_size': 32, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 4096, 'hidden_layers': 4, 'leaky_alpha': 0.1, 'lr': 5, 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'samples': 20000}
Found 16000 validated image filenames.
Found 4000 validated image filenames.
Steps per Epoch: 500, Validation Steps: 125
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 500 steps, validate for 125 steps
500/500 [==============================] - 192s 384ms/step - loss: 7916.1551 - mean_absolut_error: 65.4649 - val_loss: 5969.2841 - val_mean_absolut_error: 58.5732
Time taken: 0:03:12.276798


100%|███████████████████████████████████████████████████████████████████████████████| 72/72 [4:11:53<00:00, 209.91s/it]


### Top Ergebnisse